In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter

pd.options.display.max_rows = 1_000

### Data

In [2]:
df = pd.read_csv("../data/draft-250.csv")
df = df.sort_values("pick")
df["position"] = np.where(df["position_yahoo"].isin(["G", "D"]), df["position_yahoo"], "F")
df.head()

,team,age,name,position_yahoo,rollup,vorp,vorn,round,pick,rank,arbitrage,target,position
2,Edm,23.0,Connor McDavid,C,70.2,14.7,7.6,1.0,1.0,3.0,-2.0,True,F
0,Col,25.0,Nathan MacKinnon,C,73.6,18.1,11.0,1.0,2.0,1.0,1.0,True,F
5,Edm,25.0,Leon Draisaitl,"C,LW",67.0,12.9,7.2,1.0,3.0,6.0,-3.0,True,F
9,NYR,29.0,Artemi Panarin,LW,63.3,9.2,3.5,1.0,4.0,10.0,-6.0,True,F
3,Was,35.0,Alex Ovechkin,LW,68.5,14.4,8.7,1.0,5.0,4.0,1.0,False,F


### Player Class

In [3]:
class Player:
    
    def __init__(self, name, position, yahoo, vorp):
        self.name = name
        self.position = position
        self.yahoo = yahoo
        self.vorp = vorp 
        
    def __repr__(self):
        return f'Player("{self.name}", "{self.position}", {self.yahoo}, {self.vorp})'

### Data

In [4]:
players = []
for i, row in df.iterrows():
    player = Player(row["name"], row.position, row.pick, row.vorp)
    players.append(player)
    
players[:5]

[Player("Connor McDavid", "F", 1.0, 14.7),
 Player("Nathan MacKinnon", "F", 2.0, 18.1),
 Player("Leon Draisaitl", "F", 3.0, 12.9),
 Player("Artemi Panarin", "F", 4.0, 9.2),
 Player("Alex Ovechkin", "F", 5.0, 14.4)]

### Pool Settings

In [5]:
TEAMS = 12
SLOTS = {'F': 6, 'D': 4, 'G': 2}
BENCH = 4
PICKS = (sum(SLOTS.values()) + BENCH) * TEAMS 

### Team Class

In [6]:
class Team:
    
    def __init__(self, order, pick_strategy):
        self.players = []
        self.order = order
        self.pick_strategy = pick_strategy
        
    def __repr__(self): 
        return f"Team({self.order})"

    def pick(self, available):
        player = self.pick_strategy(available, self.players)
        self.players.append(player)
        available.remove(player)
        return player

### Strategies

In [7]:
def strategy_1(available, current_team):
    """Draft top position still required"""
    slots_to_fill = Counter(SLOTS)

    # if there are players in currently on the team, subtract those positions
    if current_team:
        slots_to_fill -= Counter([player.position for player in current_team])
        
    # if there are no more slots to fill
    if not slots_to_fill: 
        
        # return top ranked pick
        return available[0]
    
    # otherwise loop
    for player in available:
        
        # if slot still required
        if slots_to_fill.get(player.position, 0) > 0: 
            
            # pick player
            return player

def strategy_2(available, current_team):
    "Draft top Yahoo ranked picked, no matter what"
    return available[0]

def strategy_3(available, current_team):
    """Draft randomly"""
    return random.choice(available)

In [8]:
def strategy_0(available, current_team):
    """Draft VORP"""
    slots_to_fill = Counter(SLOTS)

    # if there are players in currently on the team, subtract those positions
    if current_team:
        slots_to_fill -= Counter([player.position for player in current_team])
        
    # rank
    ranked = sorted(available, key=lambda x: x.vorp, reverse=True)
        
    # if there are no more slots to fill
    if not slots_to_fill: 
        
        # return top ranked pick
        return ranked[0]
    
    # otherwise loop
    for player in ranked:
        
        # if slot still required
        if slots_to_fill.get(player.position, 0) > 0: 
            
            # pick player
            return player

### Simulate

In [55]:
def snake(low, high, x):
    k = (high - low + 1)
    return k - int(abs(x % (2*k) + low - k - 0.5))

draft_order = [snake(1, 12, x) for x in range(PICKS)]

In [56]:
teams = {order: Team(order, strategy_1) for order in range(1, TEAMS+1)}

In [65]:
# pick 
teams[11] = Team(11, strategy_0)

In [66]:
available = players.copy()
for i, team in enumerate(draft_order):
    print(i+1, team, teams[team].pick(available))

1 1 Player("Connor McDavid", "F", 1.0, 14.7)
2 2 Player("Nathan MacKinnon", "F", 2.0, 18.1)
3 3 Player("Leon Draisaitl", "F", 3.0, 12.9)
4 4 Player("Artemi Panarin", "F", 4.0, 9.2)
5 5 Player("Alex Ovechkin", "F", 5.0, 14.4)
6 6 Player("Auston Matthews", "F", 6.0, 8.2)
7 7 Player("Andrei Vasilevskiy", "G", 7.0, 16.4)
8 8 Player("Jack Eichel", "F", 8.0, 3.3)
9 9 Player("Nikita Kucherov", "F", 9.0, 3.7)
10 10 Player("Patrick Kane", "F", 10.0, 7.3)
11 11 Player("Roman Josi", "D", 29.0, 13.2)
12 12 Player("Victor Hedman", "D", 14.0, 12.5)
13 12 Player("John Carlson", "D", 21.0, 12.3)
14 11 Player("Dougie Hamilton", "D", 46.5, 11.3)
15 10 Player("Sebastian Aho", "F", 11.0, 1.6)
16 9 Player("Sidney Crosby", "F", 12.0, 4.2)
17 8 Player("Mikko Rantanen", "F", 13.0, 6.0)
18 7 Player("Elias Pettersson", "F", 15.0, -1.7)
19 6 Player("Andrei Svechnikov", "F", 16.0, 3.8)
20 5 Player("Connor Hellebuyck", "G", 17.0, 5.6)
21 4 Player("Mika Zibanejad", "F", 18.5, 7.1)
22 3 Player("Mitchell Marner", "F"

In [67]:
data = []
for team in teams:
    players_on_team = [p.name for p in teams[team].players]
    values = df[df["name"].isin(players_on_team)].groupby("position")["rollup"].mean().round().to_dict()
    values["team"] = team
    data.append(values)

In [70]:
pd.DataFrame(data).sort_values("F", ascending=False)

,D,F,G,team
0,38.0,49.0,48.0,1
1,38.0,48.0,49.0,2
2,42.0,48.0,53.0,3
10,43.0,48.0,64.0,11
11,43.0,48.0,63.0,12
9,39.0,47.0,56.0,10
4,39.0,46.0,59.0,5
5,37.0,46.0,61.0,6
7,38.0,46.0,63.0,8
8,39.0,46.0,52.0,9


In [62]:
teams[12].players

[Player("Roman Josi", "D", 29.0, 13.2),
 Player("Victor Hedman", "D", 14.0, 12.5),
 Player("Dougie Hamilton", "D", 46.5, 11.3),
 Player("Philipp Grubauer", "G", 72.0, 8.2),
 Player("Neal Pionk", "D", 80.5, 5.9),
 Player("Anton Khudobin", "G", 74.0, 1.2),
 Player("Jonathan Marchessault", "F", 97.0, -2.7),
 Player("Mike Hoffman", "F", 86.0, -3.5),
 Player("David Perron", "F", 124.0, -4.8),
 Player("Patric Hornqvist", "F", 151.0, -5.0),
 Player("Bryan Rust", "F", 178.0, -4.6),
 Player("Chris Kreider", "F", 136.5, -5.9),
 Player("Semyon Varlamov", "G", 163.0, -3.1),
 Player("Alexander Edler", "D", 261.0, -3.8),
 Player("Cam Talbot", "G", 217.0, -4.4),
 Player("Erik Cernak", "D", 182.0, -6.7)]